# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [44]:
# import libraries
import numpy as np
import nltk
import re
import pandas as pd

from sqlalchemy import create_engine

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

# # download NLTK data
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MALIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MALIK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MALIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# Load in the data from the database
engine = create_engine('sqlite:///UdacityProject2.db')
df = pd.read_sql_table('DisasterResponseClean', 'sqlite:///UdacityProject2.db')
X = df['message'].tolist()
Y = df.drop(['message', 'original', 'id'], axis = 1)
# Y.head()

In [38]:
# Get the list of target names to be used later in the script
target_names = Y.columns

# Clean the dataframe
for col in Y.columns:
    if len(df[col].unique()) == 1:
        Y.drop(col, axis = 1, inplace = True)
Y.head()

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,direct,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,direct,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,direct,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# Convert Y df to a list
Y = Y.values.tolist()

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):

    # initialize WordNetLemmatizer   
    lemmatizer = WordNetLemmatizer()

    # replace URLs with blanks
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
   
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with blanks
    for url in detected_urls:
        text = text.replace(url, "")

    # Tokenize text
    tokens = text.split()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tok = re.sub(r"[^a-zA-Z0-9]", " ", clean_tok)
        clean_tokens.append(clean_tok)  
    
    # Remove stop words
    final_clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]

    return final_clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# Create the pipeline
pipeline = Pipeline(steps = [
    ('vector',CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Load Data and perform train test split  
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
# Train the classifier
pipeline.fit(X_train, y_train)

C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale th

Pipeline(steps=[('vector',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002085D5F9F70>)),
                ('tfidf', TfidfTransformer()),
                ('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [31]:
tmp=y_pred[0].tolist()
for i in range(len(tmp)):
    if i > 0:
        tmp[i] = int(tmp[i])

print(tmp)
print(y_test[0])

test = classification_report(y_test[0], tmp)
print(test)

['news', 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
['news', 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        29
           1       1.00      0.50      0.67         6
        news       1.00      1.00      1.00         1

    accuracy                           0.92        36
   macro avg       0.97      0.83      0.87        36
weighted avg       0.92      0.92      0.90        36



In [32]:
print(y_pred)

[['news' '1' '0' ... '0' '0' '0']
 ['news' '0' '0' ... '0' '0' '0']
 ['direct' '1' '0' ... '0' '0' '0']
 ...
 ['news' '1' '0' ... '0' '0' '0']
 ['news' '1' '0' ... '0' '0' '0']
 ['direct' '1' '0' ... '0' '0' '0']]


In [29]:
# report the f1 score, precision, and recall for each output category using classification_report

def display_results(y_test, y_pred):
    
    y_test_T = [list(map(str, i)) for i in zip(*y_test)]
    y_pred_T = list(zip(*y_pred))
    
    for i in range(len(y_test_T)):       
        class_report = classification_report(y_test_T[i], y_pred_T[i])

        #print("Target Names:", target_names)
        print("Classification Report:\n", class_report)

display_results(y_test, y_pred)

Classification Report:
               precision    recall  f1-score   support

      direct       0.95      0.93      0.94      2636
        news       0.94      0.98      0.96      3330
      social       0.96      0.78      0.86       588

    accuracy                           0.94      6554
   macro avg       0.95      0.90      0.92      6554
weighted avg       0.94      0.94      0.94      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.42      0.53      1488
           1       0.84      0.94      0.89      5016
           2       0.00      0.00      0.00        50

    accuracy                           0.82      6554
   macro avg       0.51      0.46      0.47      6554
weighted avg       0.80      0.82      0.80      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      5476
           1       0.81      0.53      0.64      1078

  

C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      6036
           1       0.65      0.12      0.20       518

    accuracy                           0.93      6554
   macro avg       0.79      0.56      0.58      6554
weighted avg       0.91      0.93      0.90      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6237
           1       0.72      0.11      0.19       317

    accuracy                           0.95      6554
   macro avg       0.84      0.55      0.58      6554
weighted avg       0.95      0.95      0.94      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6389
           1       1.00      0.07      0.12       165

    accuracy                           0.98      6554
   macro avg       0.99      0.53      0.56      6554
we

C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      6175
           1       0.74      0.45      0.56       379

    accuracy                           0.96      6554
   macro avg       0.85      0.72      0.77      6554
weighted avg       0.95      0.96      0.95      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97      5876
           1       0.85      0.55      0.67       678

    accuracy                           0.94      6554
   macro avg       0.90      0.77      0.82      6554
weighted avg       0.94      0.94      0.94      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.97      6005
           1       0.80      0.36      0.50       549

    accuracy                           0.94      6554
   macro avg       0.87      0.68      0.73      6554
we

C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6481
           1       0.00      0.00      0.00        73

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      6343
           1       0.85      0.05      0.10       211

    accuracy                           0.97      6554
   macro avg       0.91      0.53      0.54      6554
weighted avg       0.97      0.97      0.96      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      6275
           1       0.88      0.20      0.33       279

    accuracy                           0.96      6554
   macro avg       0.92      0.60      0.66      6554
we

C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\MALIK\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6520
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6487
           1       0.00      0.00      0.00        67

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
we

### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
parameters = {
    'clf__n_estimators': [50, 100, 200], # dummy vals ->refine
    'clf__min_samples_split': [2, 3, 4]
}
parameters

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# feature union?

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.